In [45]:
import pandas as pd
import numpy as np

import sklearn.cluster
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances

In [25]:
seed = 42
k = 6

In [26]:
from sklearn import datasets

In [27]:
iris = datasets.load_iris()

In [28]:
X = iris.data
y = iris.target

In [31]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 50, random_state = seed)

In [36]:
kmeans = sklearn.cluster.KMeans(n_clusters = k, random_state = seed)

In [37]:
kmeans.fit(X_treino)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [40]:
centros = kmeans.cluster_centers_ 
# pd.DataFrame(centros)

,0,1,2,3
0,7.433333,3.055556,6.244444,1.977778
1,4.964516,3.377419,1.464516,0.248387
2,5.650000,2.733333,4.177778,1.311111
3,6.553846,3.169231,5.553846,2.200000
4,6.200000,2.795652,4.878261,1.652174
5,5.200000,2.366667,3.400000,1.033333


In [47]:
classificados = kmeans.predict(X_treino)
# classificados

array([2, 0, 2, 1, 0, 2, 1, 1, 1, 5, 4, 1, 1, 1, 2, 1, 2, 0, 1, 2, 4, 1,
       4, 4, 2, 2, 3, 2, 1, 5, 4, 1, 1, 5, 4, 1, 4, 1, 1, 4, 5, 3, 4, 4,
       0, 2, 1, 1, 4, 0, 1, 1, 1, 4, 3, 1, 3, 3, 1, 4, 4, 4, 4, 3, 1, 3,
       2, 3, 5, 2, 4, 1, 4, 2, 1, 2, 0, 3, 1, 5, 3, 0, 1, 3, 1, 4, 0, 3,
       2, 3, 4, 4, 4, 4, 1, 2, 2, 1, 2, 0])

In [76]:
distancias_para_cada_centro = euclidean_distances(X_treino, centros)
distancias_para_cada_centro

,0,1,2,3,4,5
0,2.769075,3.059024,0.175770,1.855920,0.919213,1.116045
1,0.414997,6.073704,3.219180,1.492486,2.273036,4.187946
2,2.575957,3.365558,0.461646,1.593441,0.753655,1.410280
3,5.680767,0.199714,3.137020,4.845360,3.989137,2.447221
4,0.586894,6.188273,3.323119,1.674416,2.385317,4.273432
...,...,...,...,...,...,...
95,2.709243,3.026104,0.488542,1.888583,0.951519,1.195361
96,3.137940,3.477840,0.933813,2.131588,1.386643,1.327487
97,5.671567,1.076081,3.424715,4.920817,4.153175,2.926128
98,2.918523,2.938819,0.290221,2.075270,1.083551,0.895669


In [235]:
df_distancias = pd.DataFrame(distancias_para_cada_centro)
df_distancias['classe'] = classificados

In [236]:
distancias_medias = df_distancias.groupby('classe').mean()
medias_por_grupo = np.diag(medias)

In [237]:
desvios = df_distancias.groupby('classe').std()
desvios_por_grupo = np.diag(desvios)

In [238]:
def make_gaussian_kernel(center, sigma):
    variance = sigma**2
    gamma = 2*(variance)
    reshaped_center = np.reshape(center, newshape = (1,-1))
    def gaussian(x):
        dist = euclidean_distances(x, reshaped_center, squared=True)
        normalization_constant = 1/(2*np.pi*variance)
        return  normalization_constant * np.exp(-(dist/gamma))
    return gaussian

In [244]:
kernels = list()
for cluster_alvo in range(k):
    alvos = X_treino[classificados==cluster_alvo]
    kernel = make_gaussian_kernel(centros[cluster_alvo], desvios_por_grupo[cluster_alvo])
    kernels.append(kernel)

In [256]:
features = list()
for kernel in kernels:
    features.append(kernel(X_treino))



    
    

In [261]:
transformed_input = pd.DataFrame(np.concatenate(features, axis = 1))

In [268]:
regression = LogisticRegression()

In [269]:
regression.fit(transformed_input, y_treino)

C:\Users\barban01\AppData\Local\Continuum\miniconda3\envs\pdg\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\barban01\AppData\Local\Continuum\miniconda3\envs\pdg\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [275]:
predicoes = regression.predict(transformed_input)

In [279]:
import sklearn.metrics
from sklearn.linear_model import LogisticRegression

In [280]:
sklearn.metrics.multilabel_confusion_matrix(y_true = y_treino, y_pred = predicoes)

array([[[69,  0],
        [ 8, 23]],

       [[65,  0],
        [17, 18]],

       [[41, 25],
        [ 0, 34]]], dtype=int64)

In [283]:
sklearn.metrics.accuracy_score(y_treino, predicoes)

0.75